# Données Exif

La plupart des appareils photos récents et téléphones portables enregistrent les photographies avec des données géographiques (longitude, latitude, mais aussi altitude). Si ces données sont lisibles avec la majorité des logiciels photos et d’explorateurs de fichiers, il est également possible d’y accéder avec Python.

Voici l'image que nous allons étudier :
![](images/thorsmork_small.jpg)

In [1]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

In [7]:
im=Image.open("images/thorsmork.jpg")

In [20]:
exif_data = im._getexif()

Répondre aux questions suivantes sur la fiche réponse :
***

1) Quel appareil photo a pris cette photographie ?
2) la largeur de l'image en pixel
3) la hauteur de l'image en pixel
4) la date et l'heure de la prise de vue
***



On peut connaitre l'ensemble des clés EXIF :
https://www.exiv2.org/tags.html


In [21]:
exif_data

{34853: {1: 'N',
  2: ((63, 1), (409847, 10000), (0, 1)),
  3: 'W',
  4: ((19, 1), (318565, 10000), (0, 1)),
  5: b'\x00',
  6: (92709, 191),
  7: ((13, 1), (18, 1), (42000, 1000)),
  12: 'K',
  13: (23, 25),
  16: 'M',
  17: (57107, 192),
  23: 'M',
  24: (57107, 192),
  29: '2018:09:03',
  31: (10, 1)},
 296: 2,
 34665: 224,
 271: 'Apple',
 272: 'iPhone 8',
 305: 'Adobe Photoshop Lightroom Classic 8.2 (Macintosh)',
 306: '2019:05:10 18:01:44',
 282: (240, 1),
 283: (240, 1),
 36864: b'0230',
 37377: (10167418, 1000000),
 37378: (1695994, 1000000),
 36867: '2018:09:03 15:18:55',
 36868: '2018:09:03 15:18:55',
 37379: (17587, 1827),
 37380: (0, 1),
 37383: 5,
 40961: 1,
 37385: 0,
 37386: (399, 100),
 41987: 0,
 41989: 28,
 41990: 0,
 37521: '756',
 37522: '756',
 41495: 2,
 33434: (1, 1150),
 33437: (18, 10),
 41729: b'\x01',
 41985: 6,
 34855: 20,
 42034: ((4183519, 1048501), (4183519, 1048501), (9, 5), (9, 5)),
 42035: 'Apple',
 42036: 'iPhone 8 back camera 3.99mm f/1.8'}

La clé "34853" n'est pas tout le temps présente puisqu'il s'agit des coordonnées (latitude, longitude) de la prise de vue, il faut donc que l'appareil photo intègre un GPS (ce qui est le cas des smartphones), si étudions cette clé "34853", nous obtenons les informations suivantes :

ligne 1 : précise que nous sommes dans l'hémisphère Nord
ligne 2 : nous avons la latitude ((47, 1), (37, 1), (29107360, 1000000)) nous avons ici une latitude en degrés, minute, seconde (ici : 47 degrés 37 minutes et 29,107360 secondes, aussi noté 47°37'29,107360")
ligne 3 : précise que nous sommes à l'ouest (W) du méridien de Greenwich
ligne 4 : nous avons la longitude ((3, 1), (25, 1), (42976570, 1000000) ici aussi la longitude est donnée en degrés, minute, seconde (ici : 3°25'42,976570")

In [20]:
exif_data[34853]

{1: 'N',
 2: ((63, 1), (409847, 10000), (0, 1)),
 3: 'W',
 4: ((19, 1), (318565, 10000), (0, 1)),
 5: b'\x00',
 6: (92709, 191),
 7: ((13, 1), (18, 1), (42000, 1000)),
 12: 'K',
 13: (23, 25),
 16: 'M',
 17: (57107, 192),
 23: 'M',
 24: (57107, 192),
 29: '2018:09:03',
 31: (10, 1)}

In [22]:
exif_data[34853][1]

'N'

In [25]:
exif_data[34853][2]

((63, 1), (409847, 10000), (0, 1))

In [27]:
exif_data[34853][2][0]

(63, 1)

cette clé correspond à 63 * 1 degré

Pour convertir des (degrés, minute, seconde aussi noté DMS) en "degrés décimaux" (DD) (unité que nous avons utilisée dans le module "cartographie"), il faut appliquer la formule suivante :
DD = degrés+(minute/60)+(seconde/3600)

ATTENTION : pour que la conversion soit correcte, il faut ajouter un signe moins devant la longitude au format DD si nous sommes situés à l'ouest du méridien de Greenwich.

On définit une fonction pour effectuer pour nous la conversion en degrés :
((63, 1), (409847, 10000), (0, 1)) correspond à :

63/1 degrés + (409847 /10000 ) minutes + (0/1) secondes  
soit un calcul à faire :


63/1 + (409847 /10000 ) *1/60 + (0/1) * 1/3600

In [29]:
def _en_degres(value): # value est ici un objet particulier : un tuple de tuples ?
    
    d0 = float(value[0][0]) # prend la valeur des degrés en nombre
    d1 = float(value[0][1]) # prend le coefficient multiplicateur
    d = d0 / d1 # on obtient les degrés

    m0 = float(value[1][0]) # prend la valeur des minutes en nombre
    m1 = float(value[1][1])
    m = m0 / m1 # on obtient les minutes

    s0 = float(value[2][0]) # prend la valeur des secondes en nombre
    s1 = float(value[2][1])
    s = s0 / s1 # on récupère les secondes

    return ( d + (m / 60.0) + (s / 3600.0) )

on essaie pour tester si la fonction "fonctionne" :

In [30]:
_en_degres(exif_data[34853][2])

63.683078333333334

autres méthodes / essais :

In [12]:
ret = {}
for tag, value in exif_data.items():
        decoded = TAGS.get(tag, tag)
        ret[decoded] = value
ret

{'GPSInfo': {1: 'N',
  2: ((63, 1), (409847, 10000), (0, 1)),
  3: 'W',
  4: ((19, 1), (318565, 10000), (0, 1)),
  5: b'\x00',
  6: (92709, 191),
  7: ((13, 1), (18, 1), (42000, 1000)),
  12: 'K',
  13: (23, 25),
  16: 'M',
  17: (57107, 192),
  23: 'M',
  24: (57107, 192),
  29: '2018:09:03',
  31: (10, 1)},
 'ResolutionUnit': 2,
 'ExifOffset': 224,
 'Make': 'Apple',
 'Model': 'iPhone 8',
 'Software': 'Adobe Photoshop Lightroom Classic 8.2 (Macintosh)',
 'DateTime': '2019:05:10 18:01:44',
 'XResolution': (240, 1),
 'YResolution': (240, 1),
 'ExifVersion': b'0230',
 'ShutterSpeedValue': (10167418, 1000000),
 'ApertureValue': (1695994, 1000000),
 'DateTimeOriginal': '2018:09:03 15:18:55',
 'DateTimeDigitized': '2018:09:03 15:18:55',
 'BrightnessValue': (17587, 1827),
 'ExposureBiasValue': (0, 1),
 'MeteringMode': 5,
 'ColorSpace': 1,
 'Flash': 0,
 'FocalLength': (399, 100),
 'WhiteBalance': 0,
 'FocalLengthIn35mmFilm': 28,
 'SceneCaptureType': 0,
 'SubsecTimeOriginal': '756',
 'SubsecTi

In [16]:
ret['GPSInfo']


{1: 'N',
 2: ((63, 1), (409847, 10000), (0, 1)),
 3: 'W',
 4: ((19, 1), (318565, 10000), (0, 1)),
 5: b'\x00',
 6: (92709, 191),
 7: ((13, 1), (18, 1), (42000, 1000)),
 12: 'K',
 13: (23, 25),
 16: 'M',
 17: (57107, 192),
 23: 'M',
 24: (57107, 192),
 29: '2018:09:03',
 31: (10, 1)}

on récupère les infos GPS utiles

In [31]:
gps_latitude = exif_data[34853][2]
gps_latitude_ref = exif_data[34853][1]
gps_longitude = exif_data[34853][4]
gps_longitude_ref = exif_data[34853][3]

gps_latitude_ref

'N'

On convertit les angles en degrés :

In [38]:
latitude = _en_degres(gps_latitude)
longitude = _en_degres(gps_longitude)



on fait attention au point cardinal de référence :


In [44]:
if gps_latitude_ref != "N":                     
                latitude = 0 - gps_latitude_degres
if gps_longitude_ref != "E":
                longitude = 0 - gps_longitude_degres


In [45]:
print(f"la latitude est :{latitude} , la longitude est {longitude} ")

la latitude est :63.683078333333334 , la longitude est -19.530941666666667 


Essai de géolocalisation :

In [48]:
import folium

macarte = folium.Map(location=[latitude,longitude],zoom_start=4)
folium.Marker([latitude,longitude]).add_to(macarte)
macarte

ça marche !

In [51]:
type(exif_data[34853])

dict

connaitre l'altitude :
ce sont les clés 5 et 6 du dictionnaire exif_data contenant le dictionnaire exif_data[34853]

la clé 5 : GPSAltitudeRef  indique si l'altitude est au dessus ou en dessous du niveau de la mer 

b'\x00'


la clé 6 : GPSAltitude donne l'altitude sous la forme d'un nombre rationnel :  

( 150, 2 ) correspond ainsi à 150/2 = 75 m

GPSAltitudeRef Indicates the altitude used as the reference altitude. If the reference is sea level and the altitude is above sea level, 0 is given. If the altitude is below sea level, a value of 1 is given and the altitude is indicated as an absolute value in the GPSAltitude tag. The reference unit is meters. Note that this tag is BYTE type, unlike other reference tags.

GPSAltitude Indicates the altitude based on the reference in GPSAltitudeRef. Altitude is expressed as one RATIONAL value. The reference unit is meters.

On peut considérer les dictionnaires comme des ensembles de paires clé: valeur, les clés devant être uniques (au sein d'un dictionnaire). 
Une paire d'accolades crée un dictionnaire vide : {}. Placer une liste de paires clé:valeur séparées par des virgules à l'intérieur des accolades ajoute les valeurs correspondantes au dictionnaire ; c'est également de cette façon que les dictionnaires sont affichés.

In [66]:
hauteur = exif_data[34853][6]
hauteur

(92709, 191)

In [71]:
hauteur_m = hauteur[0]/hauteur[1]
print (f"la hauteur en mètres vaut : {hauteur_m}" m )

la hauteur en mètres vaut : 485.3874345549738


In [76]:
ref = exif_data[34853][5]
ref

b'\x00'

\x00 signifie null , c'est un objet bytes (=octet ?)


In [78]:
type (ref)

bytes

In [85]:
ref[0] # pour accéder à la valeur du bytes en integer


0

In [88]:
int.from_bytes(ref, byteorder='big') # pour convertir un byte en integer

0

In [89]:
bytes([0]) # pour convertir un integer en byte , ne pas oublier les crochets

b'\x00'